In [ ]:
import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from constants import *

In [ ]:
df = pd.read_csv(MIMIC_CXR_SPLIT_CSV)
label_df = pd.read_csv(MIMIC_CXR_LABELS_CSV)

In [ ]:
from PIL import Image, ImageFile

In [ ]:
def check_corrupt_images(df):
        corrupt_images = []
        for idx in range(len(df)):
            row = df.iloc[idx]
            study_id = f"s{row['study_id']}"
            subject_id = f"p{row['subject_id']}"
            folder_name = subject_id[:3]
            img_path = MIMIC_CXR_JPG_DATA_DIR / "files" / folder_name / subject_id / study_id / f"{row['dicom_id']}.jpg"
            if img_path.exists():
                try:
                    with Image.open(img_path) as img:
                        img.verify()  # Verify that it is, in fact, an image
                except Exception as e:
                    print(f"Corrupt image found: {img_path} - {e}")
                    corrupt_images.append(img_path)
        return corrupt_images


In [ ]:
corrupt_images = check_corrupt_images(df)

In [ ]:
len(corrupt_images)

In [ ]:
def extract_relative_path(full_path):
    return str(full_path.relative_to('/mnt/data/MIMIC-CXR-JPG/physionet.org/files/mimic-cxr-jpg/2.1.0'))

# Create a list of relative paths
relative_paths = [extract_relative_path(path) for path in corrupt_images]

# Write the relative paths to a file
with open('IMAGE_FILENAMES', 'w') as file:
    for path in relative_paths:
        file.write(f"{path}\n")

print("File IMAGE_FILENAMES created successfully.")